In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("../../")
sys.path.append("../")

In [ ]:
from os.path import join, basename
from glob import glob
import numpy as np
import cv2
import matplotlib.pyplot as plt
from typing import List, Any

from utils import *
from estimate_alb_nrm import estimate_alb_nrm
from check_integrability import check_integrability
from construct_surface import construct_surface

# from photometric.photometric_stereo import photometric_stereo

In [ ]:
def photometric_stereo(image_dir='./SphereGray5/', path_type="column"):

    # obtain many images in a fixed view under different illumination
    print('Loading images...\n')
    [image_stack, scriptV] = load_syn_images(image_dir)
    [h, w, n] = image_stack.shape
    print('Finish loading %d images.\n' % n)

    # compute the surface gradient from the stack of imgs and light source mat
    print('Computing surface albedo and normal map...\n')
    [albedo, normals] = estimate_alb_nrm(image_stack, scriptV)


    # integrability check: is (dp / dy  -  dq / dx) ^ 2 small everywhere?
    print('Integrability checking\n')
    [p, q, SE] = check_integrability(normals)

    threshold = 0.005;
    print('Number of outliers: %d\n' % np.sum(SE > threshold))
    SE[SE <= threshold] = float('nan') # for good visualization

    # compute the surface height
    height_map = construct_surface( p, q, path_type=path_type)

    # show results
    show_results(albedo, normals, height_map, SE)

In [ ]:
photometric_stereo(image_dir='../SphereGray5/', path_type="column")

In [ ]:
photometric_stereo(image_dir='../SphereGray5/', path_type="row")

In [ ]:
photometric_stereo(image_dir='../SphereGray5/', path_type="average")